In [2]:
!pip install dnspython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 4.5 MB/s eta 0:00:00


In [3]:
import dns.resolver

In [4]:
resolver = dns.resolver.Resolver()
resolver.nameservers = ["1.1.1.1"]

In [5]:
query = resolver.resolve("google.com", "A")

In [6]:
for data in query:
  print(data)

142.250.191.46


In [7]:
query_soa = resolver.resolve("google.com", "SOA")
for data in query_soa:
  print(data)
len(query_soa)

ns1.google.com. dns-admin.google.com. 751966406 900 900 1800 60


1

In [8]:
def query_res(url, record):
  query = resolver.resolve(url, record)
  return query


In [9]:
for data in query_res("google.com", "DKIM"):
  print(data)

UnknownRdatatype: DNS resource record type is unknown.

In [ ]:
for data in query_res("google.com", "MX"):
  print(data)

In [ ]:
query_soa = resolver.resolve("cnn.com", "SOA")
for data in query_soa:
  print(data)
len(query_soa)

# Known slector

In [10]:
def fetch_dkim(selector: str, domain: str) -> list[str]:
    """
    Query DNS for the DKIM TXT record at:
        {selector}._domainkey.{domain}

    Returns a list of strings (one per TXT chunk) or raises if not found.
    """
    record_name = f"{selector}._domainkey.{domain}"
    try:
        answers = dns.resolver.resolve(record_name, "TXT")
    except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN) as e:
        raise RuntimeError(f"No DKIM record found for {record_name}") from e

    # rdata.strings is a tuple of byte-strings; join and decode each one
    txts = []
    for rdata in answers:
        # Some records come back split into multiple chunks
        full_txt = b"".join(rdata.strings).decode("utf-8")
        txts.append(full_txt)
    return txts

if __name__ == "__main__":
    sel = "default"           # your DKIM selector
    dom = "example.com"       # your domain
    try:
        records = fetch_dkim(sel, dom)
        print(f"DKIM ({sel}) for {dom}:")
        for txt in records:
            print("  ", txt)
    except RuntimeError as err:
        print(err)


DKIM (default) for example.com:
   v=DKIM1; p=


In [11]:
dkim_result = fetch_dkim("sep2018", "blackboard.com")

In [14]:
for i in dkim_result:
  display(i)

'v=DKIM1;k=rsa;g=*;p=MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAvs5qVO9zW6c82vTZNGUA9YYZKfoxtSXGdG/+QEHe0Kg9D0wIuHobngn/+NmvYSmZ2KtdnQssTF93MhXBlQ8jX3mjndj0tIaW6Snsm0+q68sJdzA7NtJBr4ljcEQRjq3jI6RQUrhs9gJ1DaKtws3SXMR8M72pQIbkuc5vkMxPCU5GPTj6TW9QweD/dZclLZ3o2AlcgONifoQY/7x2fV5GE9r55+xGB2m8yXKGeOybEkOAG9goPDp4/XQVPHfX6+Icv/OflXQ+mAuzutgyeWAe0NvYaO6NiN0I2MkcgXACsuOVwCnLs9lPkWbG9grZUEVz4wsJquAXgNQkse3eCpadIQIDAQAB'

# Check list of selectors

In [15]:
def find_dkim_selector(domain: str, selectors: list[str]) -> tuple[str, list[str]]:
    """
    Try each selector in order. Return (selector, record_texts) on first success.
    Raises RuntimeError if none found.
    """
    for sel in selectors:
        fqdn = f"{sel}._domainkey.{domain}"
        try:
            answers = dns.resolver.resolve(fqdn, "TXT")
        except (dns.resolver.NXDOMAIN, dns.resolver.NoAnswer):
            # selector not published or no TXT under that name
            continue
        # Reassemble multi-chunk TXT into full strings
        txts = [b"".join(r.strings).decode("utf-8") for r in answers]
        return sel, txts

    raise RuntimeError(f"No DKIM selector found for {domain} in list")

# example usage
common = ["default", "google", "selector1", "mail", "spf", "dkim"]
try:
    sel, record = find_dkim_selector("example.com", common)
    print(f"Found selector: {sel}\n{record}")
except RuntimeError as e:
    print(e)


Found selector: default
['v=DKIM1; p=']


In [18]:
find_dkim_selector("tamusa.edu", common)

('selector1',
 ['v=DKIM1; k=rsa; p=MIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQC79iPvavR1MznIsla6OIVk5SfHCVhxLLDwaS43NRjthA023x6QCdC+DdfpHl4+WEw6gPwhzO4nRk82KFzQBGkHt/uQvJrS7iEkAW5BNBjvzuQijfV3cdrerOkj7NqvmhVbttgM3Ptr3wJrEhtKVLqAlJCDp3HcmuVr2nALAm/GSQIDAQAB; n=1024,1453259481,1468984281'])